# AI Based Chatbot to answer DevOps Questions

### We intend to create LLM based AI Chatbot that would answer the questions about Linux

It is simple LLM bot that answers the questions only on the trained dataset unlike RAG. We will train the data on the corpus of few Linux books like Linux Bible, etc. The vectorstore to store the learnings is FAISS database.

### Technology Used
1. __LLM__: meta-llama/Llama-2-7b-chat-hf [https://huggingface.co/meta-llama/Llama-2-7b-chat-hf]
2. __VectorStore__: ___FAISS___ => FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other [https://ai.meta.com/tools/faiss/]
3. __Embeddings__: ___sentence-transformers/all-mpnet-base-v2___ [https://huggingface.co/sentence-transformers/all-mpnet-base-v2]

### Installing dependencies using pip

In [1]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes --quiet
!pip install -r requirements.txt --quiet

### Necessary Imports

In [ ]:
import os
from typing import Any
from langchain.globals import set_debug, set_verbose

### Neccessary Settings

In [3]:
import torch
from torch import cuda, bfloat16

if torch.cuda.is_available():
            torch.set_default_tensor_type(torch.cuda.HalfTensor)

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


### Loading the model

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

def load_llama(model_id: Any) -> Any:
    print(f"Loading the model {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, 
                                                 pad_token_id=tokenizer.eos_token_id
                                                )
    model.to(device)
    llama_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        return_full_text=True,
        max_new_tokens=512,
        temperature=0.7,
        task="text-generation",  # LLM task
        torch_dtype=torch.float16,
        device_map="auto",
    )
    llm = HuggingFacePipeline(pipeline=llama_pipeline)
    return llm

### Logging in to HuggingFace repo

In [5]:
import huggingface_hub
def huggingface_login() -> None:
     os.environ["HF_TOKEN"] = "YOUR_HUGGINGFACE_TOKEN"
     print("------------------------------------")
     print("Huggingface login")
     huggingface_hub.login(os.environ["HF_TOKEN"])
     print("------------------------------------\n")

### Setting the embeddings

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

def set_embeddings(model_name: Any) -> Any:
     print(f"\n------------------------------------")
     print(f"Setting the embeddings")
     embeddings = HuggingFaceEmbeddings(model_name=model_name)
     print(f"Embeddings set successfully")
     print(f"------------------------------------\n")
     return embeddings

### Loading the documents

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

def load_documents(local_directory_path: str) -> Any:
    # For PDF files
    print(f"\n------------------------------------")
    print(f"Loading PDFs from {local_directory_path}")
    loader = DirectoryLoader(local_directory_path,
                                glob='*.pdf',
                                loader_cls=PyPDFLoader)
    print(loader)
    documents = loader.load()
    print(f"Documents Loaded")
    print(f"------------------------------------\n")
    return documents

### Processing the documents

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def process_documents(documents: Any) -> Any:
    print(f"\n------------------------------------")
    print(f"Processing the documents")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                    chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    print(f"Documents processed")
    print(f"------------------------------------\n")
    return texts

### Saving to FAISS Vectorstore

In [9]:
from langchain.vectorstores import FAISS

def save_to_vectorstore(texts: Any, embeddings: Any, vectorestore_path: str) -> Any:
     print(f"\n------------------------------------")
     print(f"Saving the vectorestore to {vectorestore_path}")
     vectorstore = FAISS.from_documents(texts, embeddings)
     vectorstore.save_local(vectorestore_path)
     print(f"Vectore DB stored at {vectorestore_path}")
     print(f"------------------------------------\n")
     return vectorstore

### Setting the custom prompt template

In [10]:
from langchain import PromptTemplate


custom_prompt_template = """Use the following information to answer the user's question.
In case you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt() -> Any:
    """
    Prompt template for QA retrieval for each vectorstore
    """
    print("Setting the custom prompt")
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

### Retrieval QA Chain function

The RetrievalQAChain is a chain that combines a Retriever and a QA chain (described above). It is used to retrieve documents from a Retriever and then use a QA chain to answer a question based on the retrieved documents. [Read this for more info on RetrievalQA](https://js.langchain.com/docs/modules/chains/popular/vector_db_qa_legacy#:~:text=The%20RetrievalQAChain%20is%20a%20chain,based%20on%20the%20retrieved%20documents.)

In [11]:
from langchain.chains import RetrievalQA

def retrieval_qa_chain(llm: Any, vectorstore: Any) -> Any:
     chain = RetrievalQA.from_chain_type(llm=llm,
                                         chain_type='stuff',
                                         retriever=vectorstore.as_retriever(search_kwargs={'k': 2}), 
                                         return_source_documents=True,
                                         chain_type_kwargs={'prompt': set_custom_prompt()},
                                         verbose=False
                                         )
     return chain

### Chatbot Query

In [12]:
def chatbot(llm: Any, vectorstore: Any) -> Any:
    chain = retrieval_qa_chain(llm, vectorstore)
    exit_conditions = ("exit", "bye", "quit", ":q")
    while True:
        user_input = input("User: ")
        
        if user_input.lower() in exit_conditions:
            print("Chatbot: Thanks!")
            break
        result = chain({"query": user_input})

        response = result["result"]
        
        print("Chatbot: ", response)
        print("--------------------------------------------------\n\n")

### Sample testing. Having the debugger output on for understanding the flow of responses

In [ ]:
if __name__ == "__main__":
    set_hf_token()
    documents = load_documents("/home/sagemaker-user/content/corpus")
    texts = process_documents(documents)
    embeddings = set_embeddings('sentence-transformers/all-mpnet-base-v2')
    vectorstore = save_to_vectorstore(texts, embeddings, "/home/sagemaker-user/content/vectorstore/")

------------------------------------
Huggingface login
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/sagemaker-user/.cache/huggingface/token
Login successful
------------------------------------


------------------------------------
Loading PDFs from /home/sagemaker-user/content/corpus
Documents Loaded
------------------------------------


------------------------------------
Processing the documents
Documents processed
------------------------------------


------------------------------------
Setting the embeddings
Embeddings set successfully
------------------------------------


------------------------------------
Saving the vectorestore to /home/sagemaker-user/content/vectorstore/
Vectore DB stored at /home/sagemaker-user/content/vectorstore/
------------------------------------



In [16]:
set_debug(True)
set_verbose(True)
chatbot(load_llama("meta-llama/Llama-2-7b-chat-hf"), vectorstore)

Loading the model meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting the custom prompt


User:  What is linux


[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is linux"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is linux",
  "context": "other hand, was developed in a different context. Linux is a PC version of the Unix operating system that has been used for decades on mainframes and minicomputers and is currently the system of choice for network servers and workstations. Linux brings the \nspeed, efficiency, scalability, and flexibility of Unix to your PC, taking advantage of all the \ncapabilities that PCs can now provide.\nTechnically, Linux consists of the operating system program, referred to as the kernel,\n\nLinux ������\n \n������������������\n \n������������"
}
[llm/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm

User:  exi


[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "exi"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "exi",
  "context": "expressions, evaluating, 133–135, 443,\n620, 647–650\next2 filesystem\nchecking and repairing, 125\ndebugging, 107–110\nformatting devices as, 288–290\nlabel for, displaying, 127\nprinting block and superblock\ninformation, 124\nresizing, 356\nstoring disaster recovery data\nfor, 126\ntuning parameters of, 457–460\next3 filesystem, 16\ndebugging, 107–110\nformatting devices as, 291\nprinting block and superblock\ninformation, 124\nextended Internet services\ndaemon, 490–493\nextended regular expressions, searching\nwith, 128\nExtensible Filesystem (see XFS)\nextension command, logrotate, 254\nextension() function, gawk, 740\nExterior Gateway Pro

User:  Exit


Chatbot: Thanks!


## Final testing

In [18]:
chatbot(load_llama("meta-llama/Llama-2-7b-chat-hf"), vectorstore)

Loading the model meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting the custom prompt


User:  What is cloud computing


Chatbot:  Cloud computing refers to the practice of deploying applications and services on the internet with a cloud provider.
--------------------------------------------------




...

Chatbot:  I don't know the answer to that question. Sanjeev is a cloud and DevOps expert, not a political analyst.
--------------------------------------------------




User:  Tell me something about networking in cloud


Chatbot:  The cloud has had a significant impact on networking, as it has enabled the emergence of new networking models and technologies. For example, AWS and OpenStack provide networking services out of the box, such as VPCs, subnets, and security groups. These services have made networking a commodity much like infrastructure, allowing developers to consume networking resources on demand. Additionally, cloud networking has enabled the use of software-defined networking (SDN) and network functions virtualization (NFV) technologies, which can be used to programmatically control and automate network functions.
--------------------------------------------------




User:  Bye


Chatbot: Thanks!
